In [ ]:
!pip install transformers
!pip install -U sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import semantic_search

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2') # Alternative: sentence-transformers/LaBSE (Better according to https://docs.cohere.com/docs/multilingual-language-models)

In [ ]:
corpus = [] # Corpus to get

In [ ]:
%%time
corpus_embeddings = model.encode(corpus, convert_to_tensor=True).to('cuda')

In [ ]:
class SentenceEmbeddingsRetrieval:
    def __init__(self, corpus, corpus_embedding, model):
        self.corpus = corpus
        self.corpus_embedding = corpus_embedding
        self.model = model

    def semantic_retrieval(self, query, top_k=5):
        query_embedding = self.model.encode([query], convert_to_tensor=True).to('cuda')
        similarities = semantic_search(query_embedding, self.corpus_embedding, top_k=top_k)[0]
        results = [self.corpus[int(sim['corpus_id'])] for sim in similarities]
        return results, similarities

    def add_embedding(self, query):
        embedding = self.model.encode([query], convert_to_tensor=True).to('cuda')
        self.corpus.append(query)
        self.corpus_embedding.append(embedding)

    def remove_embedding(self, index):
        if 0 <= index < len(self.corpus):
            del self.corpus[index]
            del self.corpus_embedding[index]
        else:
            raise IndexError("Invalid index provided for removing an embedding.")


In [ ]:
retrieval = SentenceEmbeddingsRetrieval(corpus, corpus_embeddings, model)

In [ ]:
query = '' # Query
retrieval.semantic_retrieval(query)